In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
from jax import vmap
from jax import jit, lax
import matplotlib.pyplot as plt
import cmocean as cmo
import importlib

In [ ]:
from jax.config import config
config.update("jax_enable_x64", True)

In [ ]:
## import modules
import preconditioner as precond
import conjugate_gradient as cg
import pivoted_cholesky as pc
import pivoted_cholesky_ref as pc_ref # to use this script we need "torch", please comment out if not needed.
def reload():
    importlib.reload(precond)
    importlib.reload(cg)
    importlib.reload(pc)
    importlib.reload(pc_ref)
reload()

In [ ]:
import gpytorch
import torch
import linear_operator
from linear_operator.operators import (
    AddedDiagLinearOperator,
    DiagLinearOperator,
    LinearOperator,
    DenseLinearOperator,
)

In [ ]:
def generate_K(N, seed=0, noise=1e-06):
    """
    generate positive definite symmetric matrix
    """
    K = jax.random.normal(jax.random.PRNGKey(seed), (N, N))
    # K = K @ K.T + 30* jnp.eye(N) + noise*jnp.eye(N)
    # K = jnp.dot(K, K.T) + noise*jnp.eye(N)
    # K = jnp.dot(K, K.T) / N
    K = jnp.dot(K, K.T)/N
    # K += (noise+30)*jnp.eye(N) ## ??
    K += (5)*jnp.eye(N)
    K += (noise)*jnp.eye(N)
    if not is_positive_definite(K):
        raise Exception("K is not positive definite !")
    return K

In [ ]:
def is_positive_definite(matrix):
    # 行列の固有値を計算
    eigenvalues = np.linalg.eigvals(matrix)

    # 全ての固有値が正であるかをチェック
    if np.all(eigenvalues > 0):
        return True
    else:
        return False

In [ ]:
def rel_error(true, pred):
    nonzero_index = jnp.where(true != 0.)
    true = true[nonzero_index]
    pred = pred[nonzero_index]
    return jnp.mean(jnp.abs((true-pred)/true))

## 4. modified preconditioned conjugate gradient

In [ ]:
N = 100
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [ ]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [ ]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)

In [ ]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)

In [ ]:
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1)
print(rel_error(Kinvy_linalg, Kinvy))

In [ ]:
linear_operator.settings.cg_tolerance._set_value(1)
Kinvy_torch = added_diag.solve(rhs_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
linear_operator.settings.cg_tolerance._set_value(1)

### tol=1e-04

In [ ]:
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1e-04)
print(rel_error(Kinvy_linalg, Kinvy))

In [ ]:
linear_operator.settings.cg_tolerance._set_value(1e-04)
Kinvy_torch = added_diag.solve(rhs_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
# linear_operator.settings.cg_tolerance._set_value(1)

In [ ]:
N = 7000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [ ]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [ ]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)

In [ ]:
%%timeit
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1e-04)

In [ ]:
%%timeit
Kinvy_torch = added_diag.solve(rhs_torch)

In [ ]:
%%timeit
Kinvy_linalg = jnp.linalg.solve(K, rhs)

In [ ]:
print(rel_error(Kinvy_linalg, Kinvy))
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

In [ ]:
%%timeit
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=None, print_process=False, tolerance=1e-04)

### やはりpreconditionしない方がはやいのが現状

## chceking trace term

### with preconditioner

In [ ]:
N = 3000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [ ]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [ ]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, _, _ = added_diag._preconditioner()

In [ ]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1)
print(rel_error(Kinvy_linalg, Kinvy))
linear_operator.settings.cg_tolerance._set_value(1)
Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
linear_operator.settings.cg_tolerance._set_value(1)

In [ ]:
%%timeit
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=None, print_process=False, tolerance=1e-04)

In [ ]:
dKdtheta = jax.random.normal(jax.random.PRNGKey(1), (N, N))+5*jnp.eye(N)

In [ ]:
trace = jnp.einsum("ij, ij ->", Kinvy[:, :n_tridiag], jnp.einsum("ij, jk->ik", dKdtheta, zs))/n_tridiag

In [ ]:
Kinvy_linalg = jnp.linalg.inv(K)
trace_linalg = jnp.sum(jnp.diag(jnp.matmul(Kinvy_linalg, dKdtheta)))

In [ ]:
# trace_torch, _ = added_diag.inv_quad_logdet(rhs_torch, logdet=True)

In [ ]:
trace, trace_linalg, #trace_torch/2

In [ ]:
(trace_linalg-trace)/trace_linalg

### without preconditioner

In [ ]:
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=None, print_process=False, tolerance=1)
print(rel_error(Kinvy_linalg, Kinvy))

In [ ]:
dKdtheta = jax.random.normal(jax.random.PRNGKey(1), (N, N))+5*jnp.eye(N)

In [ ]:
trace = jnp.einsum("ij, ij ->", Kinvy[:, :n_tridiag], jnp.einsum("ij, jk->ik", dKdtheta, zs))/n_tridiag

In [ ]:
Kinvy_linalg = jnp.linalg.inv(K)
trace_linalg = jnp.sum(jnp.diag(jnp.matmul(Kinvy_linalg, dKdtheta)))

In [ ]:
trace, trace_linalg

In [ ]:
(trace_linalg-trace)/trace_linalg

### checking t_mat

### N = 100

In [ ]:
N = 100
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [ ]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [ ]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, _, _ = added_diag._preconditioner()

In [ ]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy, t_mat = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1, n_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy))
Kinvy_torch, t_mat_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch,num_tridiag=n_tridiag)
# Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

#### linear_operatorでは, solveの際にn_tridiagをありにするとなぜか精度が向上する

In [ ]:
t_mat.shape, t_mat_torch.numpy().shape

In [ ]:
end = 9
jnp.mean(jnp.abs(t_mat[:, :end , :end]-t_mat_torch[:, :end, :end].numpy()))

### N = 3000

In [ ]:
N = 3000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [ ]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [ ]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, _, _ = added_diag._preconditioner()

In [ ]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy, t_mat = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1, n_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy))
Kinvy_torch, t_mat_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch,num_tridiag=n_tridiag)
# Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

In [ ]:
t_mat.shape, t_mat_torch.numpy().shape

In [ ]:
end = 7
jnp.mean(jnp.abs(t_mat[:, :end , :end]-t_mat_torch[:, :end, :end].numpy()))

### Tについてはエラーが大きい: 原因はおそらくzero divisionの除外やstop updating afterの実装の有無によるalpha, betaの違いであるはず一旦sum diagだけを計算するので問題ないのでは?

### checking log determinant

In [ ]:
N = 100
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [ ]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [ ]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, precond_lt_torch, precond_logdet_torch = added_diag._preconditioner()

In [ ]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy, t_mat = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1, n_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy))
Kinvy_torch, t_mat_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch,num_tridiag=n_tridiag)
# Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

In [ ]:
inv_quad_torch, logdet_torch = added_diag.inv_quad_logdet(rhs_torch[:, -1:], logdet=True)

In [ ]:
inv_quad = jnp.dot(y[:, 0], Kinvy[: ,-1])

In [ ]:
inv_quad, inv_quad_torch

### check eigvals and eigvecs

In [ ]:
def lanczos_tridiag_to_diag(t_mat):
    eigvals, eigvectors = jnp.linalg.eigh(t_mat)
    mask = eigvals >= 0.
    eigvectors = eigvectors * jnp.expand_dims(mask, axis=-2)
    
    eigvals = eigvals.at[~mask].set(1.)
    return eigvals, eigvectors

In [ ]:
def to_dense(matrix_shape, eigenvalues, eigenvectors, funcs):
    results = [jnp.zeros(eigenvalues.shape[1:-1], dtype=eigenvalues.dtype) for _ in funcs]
    num_random_probes = eigenvalues.shape[0]
    for j in range(num_random_probes):
        eigenvalues_for_probe = eigenvalues[j]
        eigenvectors_for_probe = eigenvectors[j]
        for i, func in enumerate(funcs):
            eigenvecs_first_component = eigenvectors_for_probe[..., 0, :]
            func_eigenvalues = func(eigenvalues_for_probe)

            dot_products = (eigenvecs_first_component ** 2 * func_eigenvalues).sum(-1)
            results[i] = results[i] + matrix_shape[-1] / float(num_random_probes) * dot_products

    return results

In [ ]:
def calc_logdet(matrix_shape, t_mat, preconditioner):
    eigvals, eigvectors = lanczos_tridiag_to_diag(t_mat)
    (pinvk_logdet,) =  to_dense(K.shape, eigvals, eigvectors, [lambda x:jnp.log(x)])
    
    try:
        logdet_p = preconditioner._precond_logdet_cache
    except:
        logdet_p = 0.
    
    logdet = pinvk_logdet + logdet_p
    
    return logdet

In [ ]:
calc_logdet(K.shape, t_mat, preconditioner)

In [ ]:
eval_torch, evec_torch = linear_operator.utils.lanczos.lanczos_tridiag_to_diag(t_mat_torch)

In [ ]:
slq = linear_operator.utils.stochastic_lq.StochasticLQ()
(logdet_term,) = slq.to_dense(added_diag.matrix_shape, eval_torch, evec_torch,  [lambda x: x.log()])

In [ ]:
logdet_term

In [ ]:
inv_quad_torch, logdet_torch = added_diag.inv_quad_logdet(rhs_torch[:, -1:], logdet=True)

In [ ]:
logdet_torch

In [ ]:
det_ans = jnp.sum(jnp.log(jnp.diag(K)))
det_ans

In [ ]:
from calc_logdet import calc_logdet

#### N = 3000

In [ ]:
N = 3000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
K_torch = torch.from_numpy(np.array(K))

In [ ]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [ ]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, precond_lt_torch, precond_logdet_torch = added_diag._preconditioner()

In [ ]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy, t_mat = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1, n_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy))
Kinvy_torch, t_mat_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch,num_tridiag=n_tridiag)
# Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

In [ ]:
inv_quad_torch, logdet_torch = added_diag.inv_quad_logdet(rhs_torch[:, -1:], logdet=True)

In [ ]:
inv_quad_torch, logdet_torch

In [ ]:
jnp.dot(y[:, 0], Kinvy[: ,-1]), calc_logdet(K.shape, t_mat, preconditioner)

#### N = 5000

In [ ]:
N = 5000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
K_torch = torch.from_numpy(np.array(K))

In [ ]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [ ]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, precond_lt_torch, precond_logdet_torch = added_diag._preconditioner()

In [ ]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy, t_mat = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1, n_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy))
Kinvy_torch, t_mat_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch,num_tridiag=n_tridiag)
# Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

In [ ]:
inv_quad_torch, logdet_torch = added_diag.inv_quad_logdet(rhs_torch[:, -1:], logdet=True)

In [ ]:
inv_quad_torch, logdet_torch

In [ ]:
jnp.dot(y[:, 0], Kinvy[: ,-1]), calc_logdet(K.shape, t_mat, preconditioner)

In [ ]:
preconditioner._precond_logdet_cache, precond_logdet_torch

#### t_mat由来の誤差多少はあるが、最適化に使うだけだから大した問題ではない